<a href="https://colab.research.google.com/github/keulreobeu/deep_learning_for_the_life_sciences/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D_%EB%AA%A8%EB%8D%B8%EC%9D%98_%ED%95%B4%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 예측값 설명하기
*   예를들어 자동차, 배, 비행기를 구분하는 모델이 있다고 하자
*   그 모델이 자동차, 배, 비행기를 각각의 물체를 보고 구분하는것 뿐만아니라 배 주변의 바다, 비행기 주변의 하늘, 자동차 주변의 도로 등 다른 요소로 구분을 하면 해안가를 지나는 자동차를 구분을 못할 수 있고

*   또한 모델이 사진의 세부 사항에 지나치게 집착하여 사진속 자동차의 번호판, 배의 구명조끼등을 구분하면 수영장을 지나는 자동차를 구분하지 못할 수 있다.

*   실제 사용시 모델의 효율성이 떨어지게 된다, 즉 모델이 어떤 방식으로 예측을 하는지 아는것은 매우 중요하다.
*   즉 올바른 이유로 올바른 대답을 제공하는지 판단이 필요하다.
*   하지만 딥러닝은 입력과 출력 사이에 매우 많은 레이어 층이 있어, 이러한 모델의 복잡성은 인간의 이해력을 뛰어넘는다.
*   딥러닝 모델을 이해하기 위해 많은 방법이 발명되었다

### 돌출맵(saliency map)
*   돌출맵은 입력 이미지의 어느 부분이 출력값에 중요한지 찾는 것
*   타겟의 전체가 사용도어야 옮바르지만, 모델의 출력은 매우 복잡한 비선형 함수의 결과값이고, 실제로도 이미지의 특정 부분이 다른 부분보다 중요하게 작용하는 것이다.



In [ ]:
# 구버전 코드

import deepchem as dc
import numpy as np
from model import DRModel
from data import load_images_DR

# 데이터 로드 & 저장된 모델 불러오기
train, valid, test = load_images_DR(split='random', seed=123)
model = DRModel(n_init_kernel=32, augment=False, model_dir='test_model')
model.restore()

# 예측 진행
x = test.x
y = test.y
for i in range(10):
    prediction = np.argmax(model.predict_on_batch(x[i]))
    print('True class: %d, Predicted class: %d' % (y[i], prediction))


# 돌출맵 이미지
saliency = model.compute_saliency(x[0])
print(saliency.shape)

sal_map = np.sum(np.abs(saliency), axis=(0, 3))
sal_map -= np.min(sal_map)
sal_map /= np.max(sal_map)



# 입력값 최적화
*   돌출맵을 이용하면 자동차 사진을 식별하는 모델이 자동차를 구성하는 필셀 기반을 객체를 파악했으며, 배경을 통해 식별하지 않았다를 확인할 수 있었다.
*   머신러닝 모델은 어떻게 그 필셀이 자동차라고 결론을 지었을까?
*   일반적인 방법으론 예측 확률이 최대화하는 입려값을 찾아 확인하는것
*   하지만 모델의 특# 입력값 최적화
*   돌출맵을 이용하면 자동차 사진을 식별하는 모델이 자동차를 구성하는 필셀 기반을 객체를 파악했으며, 배경을 통해 식별하지 않았다를 확인할 수 있었다.
*   머신러닝 모델은 어떻게 그 필셀이 자동차라고 결론을 지었을까?
*   일반적인 방법으론 예측 확률이 최대화하는 입려값을 찾아 확인하는것
*   하지만 모델의 입력에 변화를 주며 특징을 추출하는 과정(ex:Pooling)에서 나온 이미지는 사람의 눈에는 의미있는 이미지는 아니다.

### 교재 예시 정리
-   DNA의 전사부위에 결합을 하는지 예측하는 모델
*   목적: 모델이 어떤 패턴(서열/픽셀)을 “중요한 특징”으로 사용하는지 확인하기 위해 입력값을 직접 조정.

#### 방법

    예측 확률을 최대화하는 입력값 탐색
    → 입력 데이터를 반복적으로 변형하면서, 목표 클래스의 예측 확률이 올라가면 그 변형을 유지, 그렇지 않으면 버림.

    반복적 강화(gradient ascent 유사)
    → 여러 번 반복하면, 예측 확률을 극대화하는 “핵심 패턴”이 강조된 입력값이 남음.

    돌출맵(saliency map) 병행 사용
    → 어떤 위치(서열, 픽셀 등)가 모델의 출력을 강하게 바꾸는지 시각적으로 확인.

*   요약: 입력 최적화 = 입력 데이터 자체를 조정하면서 “이 모델이 중요하다고 생각하는 부분”을 역으로 드러내는 과정입니다.
*   중간 과정에 입력데이터를 반복적으로 변형하여 예측확률이 올라가는 지점을 점점 고도화 하였는데 이는 예측을 위한 학습이 아닌 어떠한 규칙을 통해 예측을 하는지 확인하는 과정


# 불확실성 예측하기
*   머신러닝 모델의 예측이 정확하다고 해도 예측의 정확도가 어느 정도 되야 정확하다고 할수 있을까?
*   과학은 유효숫자처럼 모든 값의 불확실성을 확안하는 것을 원한다.
*   만약 예측값이 1352라면 그값은 1351과 1353사이의 값으로 해석될 수 있다.
*   예를들어 용해도 특정 모델이 있다. 이 모델의 RMS값이 0.396이 나온다. 그럲다면 이 값을 모델의 불확실성 값으로 사용할 수 있을까?
*   만약 테스트 데이터 셋이 모든 입력값을 대변하거나 예측 오류가 같은 분포를 가진다면 그렇게 할 수 있지만 이는 현실적이지 않다.
*   그러기 때문에 모델의 예측 오류를 유발하는 요소는 신중히 고려되어야 한다.
*   용해도 모델을 학습한 데이터의 경우 고르게 분포된 데이터를 가지고 있음에도 불구하고 열개의 분자를 반복적으로 예측을 했을때 그 결과는 매번 다른값을 가진다.
*   이를 인식론적 불확실성이라 하며, 학습한 데이터를 통한 모델의 성능 측정은 어느 모델이 최고인지 판단할 수 없다.
*   다른 대안으로 단일 모델에 어려 드롭아웃을 적용해 여러 번 예측하는 방법이 있다.
*   이를 통해 인식론적 불확실성에 대해 추정할 수 있다.
*   하지만 모델에 매개변수가 거의 없는 학습모델에 사용한다면 최적의 매개변수가 특정한 값이 될 확률이 높아 특정 학습 데이터를 대변하지 못한다.
*   이러한 dropout을 통해 출력요소의 평균값과 표준 편차의 추정값을 얻을 수 있다.


# 해석 가능성, 설명 가능성, 실제 결과
*   잘못된 예측의 파장이 클수록 모델의 작동방식을 이해하는 것이 중요하다.
*   약물 선별 모델의 경우 모델의 정확도가 낮아도 예측값이 평균보다 높으면 충분하지만, 질병의 진단 및 치료법 추천은 환자의 삶이 예측결과에 큰 영향을 받기 떄문에 예측값의 신뢰도가 높아야 한다.
*   또한 이상적인 모델의 경우 병을 진단할 뿐 아니라 진단을 뒷바침하는 증거도 제공해야 모델이 정확하게 진단했는지 판단할 수 있을 것이기 때문이다.
*   사람은 종종 심리학이나 경제학에 의존하기 때문에 예측에 대한 상관계수나 정확도로는 충분하지 않다.
*   그리하여 예측값을 누가 보는지, 어떻게 해석해야 되는지, 실제 효과는 어떤지를 항상 생각해야한다.